# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,winnemucca,40.9730,-117.7357,-6.97,86,0,2.57,US,2025-01-08
1,1,port-aux-francais,-49.3500,70.2167,5.85,73,28,9.21,TF,2025-01-08
2,2,puerto natales,-51.7236,-72.4875,11.25,82,75,7.20,CL,2025-01-08
3,3,fort bragg,35.1390,-79.0060,-1.72,65,0,3.09,US,2025-01-08
4,4,punta arenas,-53.1500,-70.9167,12.06,76,75,4.63,CL,2025-01-08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

16


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,grytviken,-54.2811,-36.5092,6.92,55,30,0.72,GS,2025-01-08
50,50,al jawf,29.5000,38.7500,11.10,43,0,3.11,SA,2025-01-08
61,61,dawson creek,55.7666,-120.2362,3.80,65,0,6.69,CA,2025-01-08
290,290,santa maria del oro,25.9333,-105.3667,9.16,58,90,1.89,MX,2025-01-08
317,317,katsuura,35.1333,140.3000,6.68,66,30,8.37,JP,2025-01-08


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
29,grytviken,GS,-54.2811,-36.5092,55,
50,al jawf,SA,29.5000,38.7500,43,
61,dawson creek,CA,55.7666,-120.2362,65,
290,santa maria del oro,MX,25.9333,-105.3667,58,
317,katsuura,JP,35.1333,140.3000,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'filter': '',
    'bias': '',
    'apiKey': geoapify_key,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        name_address = requests.get(base_url, params=params)
        name_address = name_address.json()
        
        # Debugging: Print the response to verify structure
        print(f"Response for {row['City']}: {json.dumps(name_address, indent=2)}")

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if name_address["features"]:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except (KeyError, IndexError) as e:
        print(f"Error for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
Response for grytviken: {
  "type": "FeatureCollection",
  "features": []
}
grytviken - nearest hotel: No hotel found
Response for al jawf: {
  "type": "FeatureCollection",
  "features": []
}
al jawf - nearest hotel: No hotel found
Response for dawson creek: {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "Comfort Inn",
        "country": "Canada",
        "country_code": "ca",
        "state": "British Columbia",
        "county": "Peace River Regional District",
        "city": "Dawson Creek",
        "postcode": "V1G 1Y9",
        "street": "Alaska Highway",
        "housenumber": "1200",
        "lon": -120.2336243,
        "lat": 55.7611403,
        "state_code": "BC",
        "formatted": "Comfort Inn, 1200 Alaska Highway, Dawson Creek, BC V1G 1Y9, Canada",
        "address_line1": "Comfort Inn",
        "address_line2": "1200 Alaska Highway, Dawson Creek, BC V1G 1Y9, Canada",
        "cat

,City,Country,Lat,Lng,Humidity,Hotel Name
29,grytviken,GS,-54.2811,-36.5092,55,No hotel found
50,al jawf,SA,29.5000,38.7500,43,No hotel found
61,dawson creek,CA,55.7666,-120.2362,65,Comfort Inn
290,santa maria del oro,MX,25.9333,-105.3667,58,No hotel found
317,katsuura,JP,35.1333,140.3000,66,潮騒の宿


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)